# Contextual Relevancy

Contextual Relevancy เป็นเมทริกที่ใช้วัด RAG โดยการประเมินความเกี่ยวข้องของข้อมูลที่แสดง retrieval_context กับ input ที่กำหนด 

### Required Arguments

- input
- actual_output
- retrieval_context

note:
เทคนิคนี้คล้ายกับ ContextualPrecisionMetric 

In [2]:
from langchain_openai import AzureChatOpenAI
from deepeval.models.base_model import DeepEvalBaseLLM
import sys
sys.path.append('/opt/project/src/evaluate_llm/')
from api_key_config import settings
import os

os.environ["OPENAI_API_VERSION"] = settings.OPENAI_API_VERSION
os.environ["OPENAI_API_KEY"] = settings.OPENAI_API_KEY
os.environ["AZURE_OPENAI_ENDPOINT"] = settings.AZURE_OPENAI_ENDPOINT

class AzureOpenAI(DeepEvalBaseLLM):
    def __init__(
        self,
        model
    ):
        self.model = model

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        chat_model = self.load_model()
        return chat_model.invoke(prompt).content

    async def a_generate(self, prompt: str) -> str:
        chat_model = self.load_model()
        res = await chat_model.ainvoke(prompt)
        return res.content

    def get_model_name(self):
        return "Custom Azure OpenAI Model"

# Replace these with real values
custom_model = AzureChatOpenAI(
    deployment_name="gpt-35-turbo",
)
azure_openai = AzureOpenAI(model=custom_model)

In [3]:
from deepeval import evaluate
from deepeval.metrics import ContextualRelevancyMetric
from deepeval.test_case import LLMTestCase

# Replace this with the actual output from your LLM application
actual_output = "We offer a 30-day full refund at no extra cost."

# Replace this with the actual retrieved context from your RAG pipeline
retrieval_context = ["All customers are eligible for a 30 day full refund at no extra cost."]

metric = ContextualRelevancyMetric(
    threshold=0.7,
    model=azure_openai,
    include_reason=True
)
test_case = LLMTestCase(
    input="What if these shoes don't fit?",
    actual_output=actual_output,
    retrieval_context=retrieval_context
)

metric.measure(test_case)
print(metric.score)
print(metric.reason)

# or evaluate test cases in bulk
evaluate([test_case], [metric])

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Output()

0.0
The score is 0.00 because the context about the 30 day full refund policy is irrelevant to the input about the fit of the shoes.
Evaluating test cases...
Event loop is already running. Applying nest_asyncio patch to allow async execution...




Metrics Summary

  - ❌ Contextual Relevancy (score: 0.0, threshold: 0.7, strict: False, evaluation model: Custom Azure OpenAI Model, reason: The score is 0.00 because the context about the refund policy is irrelevant to the question about the fit of the shoes., error: None)

For test case:

  - input: What if these shoes don't fit?
  - actual output: We offer a 30-day full refund at no extra cost.
  - expected output: None
  - context: None
  - retrieval context: ['All customers are eligible for a 30 day full refund at no extra cost.']


Overall Metric Pass Rates

ContextualRelevancyMetric: 0.00% pass rate




/usr/local/lib/python3.9/site-packages/portalocker/utils.py:218: UserWarning: timeout has no effect in blocking mode
  warnings.warn(


✅ Tests finished! Run "deepeval login" to view evaluation results on the web.

[TestResult(success=False, metrics=[<deepeval.metrics.contextual_relevancy.contextual_relevancy.ContextualRelevancyMetric object at 0x7fec97f5a6d0>], input="What if these shoes don't fit?", actual_output='We offer a 30-day full refund at no extra cost.', expected_output=None, context=None, retrieval_context=['All customers are eligible for a 30 day full refund at no extra cost.'])]

There are five optional parameters when creating a ContextualRelevancyMetricMetric:

- [Optional] threshold: a float representing the minimum passing threshold, defaulted to 0.5.
- [Optional] model: a string specifying which of OpenAI's GPT models to use, OR any custom LLM model of type DeepEvalBaseLLM. Defaulted to 'gpt-4o'.
- [Optional] include_reason: a boolean which when set to True, will include a reason for its evaluation score. Defaulted to True.
- [Optional] strict_mode: a boolean which when set to True, enforces a binary metric score: 1 for perfection, 0 otherwise. It also overrides the current threshold and sets it to 1. Defaulted to False.
- [Optional] async_mode: a boolean which when set to True, enables concurrent execution within the measure() method. Defaulted to True.

### How Is It Calculated?

Contextual Relevancy = Number of Relevant Statements/Total Number of Statements

ถึงแม้ว่าจะคล้ายกับวิธีการคำนวณ AnswerRelevancyMetric แต่ ContextualRelevancyMetric จะใช้ LLM เพื่อดึงข้อความทั้งหมดที่ปรากฏใน retrieval_context ก่อน จากนั้นใช้ LLM เดียวกันในการจำแนกว่าแต่ละข้อความเกี่ยวข้องกับ input หรือไม่